# Evaluation
- compare verb/nouns 
- look at frequency of repeated words
- number of stop words
- free form prompts for empirical evaluation
- look for common beatles themes (heartbreak, girl, dancing,)
- syllable counts
- compare 

In [1]:
# !pip install transformers
# !pip install torch

In [72]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [73]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length):
    model_path = "results"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    return(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [159]:
test = pd.read_parquet('testing_lyrics.parquet').reset_index(drop=True)
test.head()

,lyrics
0,"Don't let me down, don't let me down Don't let..."
1,I get high when I see you go by My oh my When ...
2,[All too much!] It's all too much It's all to...
3,"I love you, 'cause you tell me things I want t..."
4,Is there anybody going to listen to my story A...


In [181]:
test['input'] = test['lyrics'].apply(lambda x: ' '.join(x.split()[:50])) #50 tokens
test['prediction'] = test['input'].apply(lambda x: generate_text(x,200))
# test.to_parquet('predictions.parquet')

In [190]:
preds = pd.read_parquet('predictions.parquet')

In [191]:
preds['lyrics_end'] = preds['lyrics'].apply(lambda x: ' '.join(x.split()[50:100]))
preds['prediction_end'] = preds['prediction'].apply(lambda x: ' '.join(x.split()[50:100]))

In [202]:
preds['lyrics_end'][13]

"the end I'm a loser And I lost someone who's near to me I'm a loser And I'm not what I appear to be Although I laugh and I act like a clown Beneath this mask I am wearing a frown My tears are falling like rain from the sky"

In [203]:
preds['prediction_end'][13]

"the end I'm a loser And I'm not what I appear to be Of all the love I have won or have lost There is one love I should never have crossed She was a girl in a million, my friend I should have known she would win in the"

## BLEAU Score

In [214]:
print(preds['lyrics_end'][0].lower().replace(',','').replace('(','').replace(')','').replace("'",'').split(' '))
print(preds['prediction_end'][0].lower().replace(',','').replace('(','').replace(')','').replace("'",'').split(' '))

['let', 'me', 'down', 'dont', 'let', 'me', 'down', 'dont', 'let', 'me', 'down', 'im', 'in', 'love', 'for', 'the', 'first', 'time', 'dont', 'you', 'know', 'its', 'gonna', 'last', 'its', 'a', 'love', 'that', 'lasts', 'forever', 'its', 'a', 'love', 'that', 'has', 'no', 'past', 'seeking', 'past', 'dont', 'let', 'me', 'down', 'dont', 'let', 'me', 'down', 'dont', 'let', 'me']
['let', 'me', 'down', 'dont', 'let', 'me', 'down', 'dont', 'let', 'me', 'down', 'and', 'from', 'the', 'first', 'time', 'that', 'she', 'really', 'done', 'me', 'oh', 'she', 'done', 'me', 'yeah', 'she', 'done', 'me', 'oh', 'yeah', 'oh', 'yeah', 'oh', 'yeah', 'oh', 'yeah', 'imagine', 'im', 'in', 'love', 'with', 'you', 'its', 'easy', 'cos', 'i', 'know', 'ive', 'imagined']


In [218]:
len(preds)

34

In [227]:
import statistics
from nltk.translate.bleu_score import sentence_bleu

reference = [['hello', 'not', 'my', 'big']]
candidate = ['this', 'is', 'a', 'test']
score = sentence_bleu(reference, candidate)
print(score)

0


In [228]:
import statistics
from nltk.translate.bleu_score import sentence_bleu

scores=[]

for i in range(len(preds)):
    reference = [preds['lyrics_end'][i].lower().replace(',','').replace('(','').replace(')','').replace("'",'').split(' ')]
    candidate = preds['prediction_end'][i].lower().replace(',','').replace('(','').replace(')','').replace("'",'').split(' ')
    scores.append(sentence_bleu(reference, candidate))

scores

[0.2704451468820861,
 0.7573260996605835,
 0.4185180195193637,
 1.6374957314724148e-78,
 0.3569804363478891,
 1.0,
 0.8162735195826227,
 0.054285521517746294,
 0.7255295110702159,
 0.9381103447512052,
 1.0,
 0.36483085104667456,
 0.7730575253950128,
 0.26611662785390383,
 0.22704626973856634,
 1.0,
 0.53487753050603,
 1.0,
 1.0,
 0.15702128402250726,
 0.8576907222981509,
 1.072268715173723e-231,
 0.3499287218249928,
 0.958740481558872,
 3.6723267991733874e-155,
 0.3830543913117282,
 0.1974438117016396,
 0.24575159036693417,
 0.15186599422096483,
 1.0,
 3.4486983131945593e-155,
 0.896849229036064,
 0.316937673914309,
 0.6252029912577429]